<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Ecorces_Arbres/Id_Ecorces-Analyse_Erreurs-Colab" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter) 
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous quand vous faites des retours en arrière, cela peut réinitialiser certaines variables.

SVP, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

# Identification d'arbres à partir de leur écorce
## Réseau convolutif, apprentissage par transfert et amplification des données

In [ ]:
!rm -r /content/modeles

### Inspiration et droits d'auteur

Ce laboratoire s'inspire de plusieurs oeuvres en logiciels libres qui ont été transformées dont:

<a href="https://www.tensorflow.org/tutorials/images/transfer_learning" target='_blank'>Transfer learning and fine-tuning</a> - site Google / Tutoriels TensorFlow

<a href="https://www.tensorflow.org/tutorials/images/data_augmentation" target='_blank'>Data augmentation</a> - site Google / Tutoriels TensorFlow

##### Copyright (c) 2017,  François Chollet  
##### Copyright (c) 2019-2022, The TensorFlow Authors.
##### Copyright (c) 2022, Claude Coulombe

Le contenu de cette page est sous licence <a href="https://creativecommons.org/licenses/by/4.0/deed.fr" target='_blank'>Creative Commons Attribution 4.0 (CC BY 4.0)</a>,<br/>et les exemples de code sont sous <a href="https://www.apache.org/licenses/LICENSE-2.0" target='_blank'>licence Apache 2.0</a>.

#### Données

Les données sur les écorces d'arbres proviennent de <a href="https://data.mendeley.com/research-data/?search=barknet">BarkNet<sup>1</sup></a>, une banque en données ouvertes sous licence MIT de 23 000 photos d'écorces d'arbres en haute résolution prises avec des téléphones intelligents par une équipe d'étudiants et de chercheurs du <a href="https://www.sbf.ulaval.ca/" target='_blank'>Département des sciences du bois et de la forêt de l'Université Laval</a> à Québec.</p>

# Analyse d'erreur

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>

**Note** : Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 1

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)

# Note: Retrait du comportement déterministe
# à cause de keras.layers.RandomContrast(...)
# dont il n'existe pas de version déterministe
# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Acquisition des données

L'analyse d'erreur requiert l'examen des données d'entraînement, il faut donc les obtenir.


Notez qu'en raison, des limites imposées par Colab, nous avons échantillonné 1.5 Go de données sur les 32 Go de données initiales de BarkNet. 

Aussi, nous n'avons pas inclus Acer platanoides (2), Pinus rigida (15) et Populus grandidentata (18) car il n'y a pas suffisamment d'images dans ces catégories pour obtenir des résultats significatifs.


In [ ]:
data_ecorces = {
    'BOJ': 1, 
    'BOP': 2,
    'CHR': 3,
    'EPB': 4,
    'EPN': 5,
    'EPO': 6,
    'EPR': 7,
#   'ERB': 8,  # Pas assez de spécimens - seulement 1
    'ERR': 8, 
    'ERS': 9, 
    'FRA': 10, 
    'HEG': 11,  
    'MEL': 12,  
    'ORA': 13,  
    'OSV': 14, 
#   'PEG': 15, # Pas assez de spécimens - seulement 3
    'PET': 15, 
    'PIB': 16, 
#   'PID': 17, # Pas assez de spécimens - seulement 4
    'PIR': 17, 
    'PRU': 18, 
    'SAB': 19,  
    'THO': 20, 
}

noms_arbres = {
    1  : '\emph{Betula alleghaniensis} - Bouleau jaune - Yellow birch',
    2  : '\emph{Betula papyrifera} - Bouleau à papier - White birch',
    3  : '\emph{Quercus rubra} - Chêne rouge - Northern red oak',
    4  : '\emph{Picea glauca} - Épinette blanche - White spruce',
    5  : '\emph{Picea mariana} - Épinette noire - Black spruce',
    6  : '\emph{Picea abies} - Épinette de Norvège - Norway spruce',
    7  : '\emph{Picea rubens} - Épinette rouge - Red spruce',
  # 8  : '\emph{Acer platanoides} - Érable de Norvège - Norway maple',
    8  : '\emph{Acer rubrum} - Érable rouge - Red maple',
    9  : '\emph{Acer saccharum} - Érable à sucre - Sugar maple',
    10 : "\emph{Fraxinus americana} - Frêne d'Amérique - White ash",
    11 : '\emph{Fagus grandifolia} - Hêtre à grandes feuilles - American beech',
    12 : '\emph{Larix laricina} - Mélèze - Tamarack',
    13 : "\emph{Ulmus americana} - Orme d'Amérique - American elm",
    14 : '\emph{Ostrya virginiana} - Ostryer de Virginie - American hophornbeam',
  # 15: '\emph{Populus grandidentata} - Peuplier à grandes dents - Big-tooth aspen',
    15 : '\emph{Populus tremuloides} - Peuplier faux tremble - Quaking aspen',
    16 : '\emph{Pinus strobus} - Pin blanc - Eastern white pine',
  # 17 : '\emph{Pinus rigida} - Pin rigide - Pitch pine',
    17 : '\emph{Pinus resinosa} - Pin rouge - Red pine',
    18 : '\emph{Tsuga canadensis} - Pruche du Canada - Eastern Hemlock',
    19 : '\emph{Abies balsamea} - Sapin Baumier - Balsam fir',
    20 : '\emph{Thuja occidentalis} - Thuya occidental - Northern white cedar',
}

dict_no_arbres_ID = {
    1  : 'BOJ', 
    2  : 'BOP',
    3  : 'CHR',
    4  : 'EPB',
    5  : 'EPN',
    6  : 'EPO',
    7  : 'EPR',
  # 8  : 'ERB',  # Pas assez de spécimens - seulement 1
    8  : 'ERR', 
    9  : 'ERS', 
    10 : 'FRA', 
    11 : 'HEG',  
    12 : 'MEL',  
    13 : 'ORA',  
    14 : 'OSV', 
  # 15 : 'PEG', # Pas assez de spécimens - seulement 3
    15 : 'PET', 
    16 : 'PIB', 
  # 17 : 'PID', # Pas assez de spécimens - seulement 4
    17 : 'PIR', 
    18 : 'PRU', 
    19 : 'SAB',  
    20 : 'THO', 
}

print("Code exécuté")

In [ ]:
# Dictionnaire Python des URL qui pointent vers des données sur Google Doc

data_zip_urls_dict = {
   "BOJ":"https://drive.google.com/file/d/1d2zxg2pt5S8UJIK-E7IuWfGN0d1kxxMw/view?usp=sharing",
   "BOP":"https://drive.google.com/file/d/12cg6UO4HLnjk5fE_KXtrgdC2s8uGh4Zp/view?usp=sharing",
   "CHR":"https://drive.google.com/file/d/1Nq19-I-Q577KXMTFrkhlJDhMfclh0cWn/view?usp=sharing",
   "EPB":"https://drive.google.com/file/d/1K_Ncw8VEiuDZ_iJDbYToMq-GO5dzKHns/view?usp=sharing",
   "EPN":"https://drive.google.com/file/d/1S309DYmg76SrIA89aVQWXCMwm6CzhN8b/view?usp=sharing",
   "EPO":"https://drive.google.com/file/d/1fTKEcpYgmRg4spUpcH0FAiAnoRgANafL/view?usp=sharing",
   "EPR":"https://drive.google.com/file/d/1qRhtZ8LZjH_45fxetG7swg3ok3znk8CJ/view?usp=sharing",
#   "ERB":"https://drive.google.com/file/d/1ighbGniKAT_GrPm4RtsIAuN1STg9sjR9/view?usp=sharing", # Assez de données?
   "ERR":"https://drive.google.com/file/d/1rEo1thMNJTgFeTzTOfI11_FPSqMgbHSL/view?usp=sharing",
   "ERS":"https://drive.google.com/file/d/1ts-t7bOH9DfKj0q0v35nMgKHgVT0ZjyG/view?usp=sharing",
   "FRA":"https://drive.google.com/file/d/1yLacRGW7JtlFWV5asEXHpAToClL38D64/view?usp=sharing",
   "HEG":"https://drive.google.com/file/d/1zoJKEIrsCD1XxglgPJkEygumev1xRQ3U/view?usp=sharing",
   "MEL":"https://drive.google.com/file/d/1Wdy3DDnWfUysXjcIFFq12UFW7tlTYDT2/view?usp=sharing",
   "ORA":"https://drive.google.com/file/d/19_oYwCAaPfP6vMuqUnAzIQAa39Brxhfi/view?usp=sharing",
   "OSV":"https://drive.google.com/file/d/1VJCCZN1iwBK2Nzh_PHC9xvw63xiLuXXI/view?usp=sharing",
#   "PEG":"https://drive.google.com/file/d/1YUWH4IaTnmcoIAavZq8HyXByJxO7_zBg/view?usp=sharing", # Assez de données?
   "PET":"https://drive.google.com/file/d/13bMkvr_1mRz1TuOcX8-c-LfTSIsNKrve/view?usp=sharing",
   "PIB":"https://drive.google.com/file/d/17J9g1xm6-ji52k2pgJr7mUrJdS1ASSqP/view?usp=sharing",
#   "PID":"https://drive.google.com/file/d/12xswrf4pDmTAcYZDAY9D-0HniLjGJCxp/view?usp=sharing", # Assez de données?
   "PIR":"https://drive.google.com/file/d/1qny4meuoT-HYZ_KTyPQbQnzLhebkgkfU/view?usp=sharing",
   "PRU":"https://drive.google.com/file/d/1xQWHQvIbwRRBoi2F27q22_drUeM8m3S8/view?usp=sharing",
   "SAB":"https://drive.google.com/file/d/1ol2mlYAz5bMfQkwqcnxhCOg4avftYtRe/view?usp=sharing",
   "THO":"https://drive.google.com/file/d/1_mI0saGpfxb4wnhElCzxg0WU4OiFHkfP/view?usp=sharing",
  
}

data_zip_urls_dict

In [ ]:
# Création des répertoires de données
# Nous allons créer un répertoire de base `data` et des répertoires pour les données 
# d'entrainement, de validation et de test pour chaque étiquette cible.
# Enfin, un répertoire `modeles` pour mémoriser les modèles entraînés

try:
    os.mkdir("/content/data/")
except OSError:
    pass
try:
    os.mkdir("/content/lab_ecorces/")
except OSError:
    pass
try:
    os.mkdir("/content/modeles/")
except OSError:
    pass


In [ ]:
# Demande d'autorisation pour télécharger les données sur Google Drive 
# Référence: https://colab.research.google.com/notebooks/io.ipynb

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import shutil
import zipfile

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Téléchargement et décompression des données

nombre_classes = 0
for arbre_id in data_zip_urls_dict.keys():
    url = data_zip_urls_dict[arbre_id]
    id_fichier = url.split('/')[5]
    fichier = drive.CreateFile({'id':id_fichier})
    nom_fichier = arbre_id + ".zip"
    # télécharger le fichier nom_fichier
    fichier.GetContentFile("/content/data/" + nom_fichier)
    print("Fichier " + nom_fichier + " téléchargé")
    zip_ref = zipfile.ZipFile("/content/data/" + nom_fichier, 'r')
    zip_ref.extractall("/content/data")
    zip_ref.close()
    print("Fichier " + nom_fichier + " décompressé")
    try:
        os.remove("/content/data/"+nom_fichier)
        print("Fichier " + nom_fichier + " effacé")
    except:
        print("?")
    nombre_classes += 1
shutil.rmtree('/content/data/__MACOSX')
print("nombre_classes:",nombre_classes)

### Répartition des données

In [ ]:
# Installation des bibliothèques Python `split-folders` et `tqdm`

!pip3 install split-folders tqdm

In [ ]:
# Répartition des données d'entraînement, de validation et de tests

import splitfolders
import pathlib

#### répertoire des données en entrée et des données une fois réparties
repertoire_entree = "/content/data"
repertoire_donnees_reparties = "/content/lab_ecorces"
# => train, val, test

nombre_images = len(list(pathlib.Path(repertoire_entree).glob('*/*.jpg')))
print("Nombre total d'images:",nombre_images)

splitfolders.ratio(repertoire_entree, output=repertoire_donnees_reparties, seed=42, ratio = (0.80, 0.15, 0.05))

print("\nRépartition des données terminée!")

### Téléverser et décompresser un modèle Keras sauvegardé sur un poste local

1) & 2) Vous allez téléverser le modèle d'identification d'écorces entraîné précédemment.

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier.png"/>

3) La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher le modèle d'identification d'écorces modele_....zip que vous avez sauvegardé sur votre ordinateur local.

Attention! Assurez-vous que le fichier est entièrement téléversé et que l'icône d'état du téléversement (un cercle jaune) disparaisse. 

<img src="https://cours.edulib.org/asset-v1:Cegep-Matane+VAERN.1FR+P2021+type@asset+block@Colab_Importer_Fichier-3.png"/>

4) Décompressez le fichier modele_....zip en exécutant la commande ci-dessous:

In [ ]:
!unzip /content/modele_*.zip -d /content/modeles && rm /content/modele_*.zip

In [ ]:
import os
import numpy as np
import keras
print("Version de Keras:",keras.__version__)
import tensorflow as tf
print("Version de TensorFlow :",tf.__version__)

In [ ]:
chemin_modele_sauvegarde = "/content/modeles/"

modele_de_transfert = tf.keras.models.load_model(chemin_modele_sauvegarde)

In [ ]:
modele_de_transfert.summary()

#### Prédiction sur les données d'entraînement

In [ ]:
REPERTOIRE_ENTRAINEMENT = "/content/lab_ecorces/train/"

TAILLE_LOT = 32
HAUTEUR_IMAGE = 150
LARGEUR_IMAGE = 150
TAILLE_IMAGE = (HAUTEUR_IMAGE, LARGEUR_IMAGE)
NOMBRE_CANAUX = 3

AUTOTUNE = tf.data.AUTOTUNE

couches_amplification = tf.keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomFlip("vertical"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.3),
    keras.layers.RandomContrast(0.3),
])

couches_normalisation = keras.Sequential([
    keras.layers.Resizing(HAUTEUR_IMAGE,LARGEUR_IMAGE),
    keras.layers.Rescaling(1./255)
])

def pretraitement(jeu_donnees, melanger=False, amplifier=False):

    if melanger:
        jeu_donnees = jeu_donnees.shuffle(1000)
        
    # Amplifier seulement les données d'entraînement
    if amplifier:
        jeu_donnees = jeu_donnees.map(lambda x, y: (couches_amplification(x,training=True), y),
                                      num_parallel_calls=AUTOTUNE
                                     )
        
    # Normaliser les jeux de données
    jeu_donnees = jeu_donnees.map(lambda x, y: (couches_normalisation(x), y),
                                  num_parallel_calls=AUTOTUNE
                                 )
    
    # Utiliser des tampons de préextraction sur tous les jeux de données
    return jeu_donnees.prefetch(buffer_size=AUTOTUNE)

print("Fonction de prétraitement prête!")


In [ ]:
donnees_entrainement = tf.keras.utils.image_dataset_from_directory(REPERTOIRE_ENTRAINEMENT,
                                                                   batch_size=TAILLE_LOT,
                                                                   image_size=TAILLE_IMAGE,
                                                                   shuffle=False)

donnees_entrainement_normalisees = pretraitement(donnees_entrainement,
                                                 melanger=False,
                                                 amplifier=False)


##### Inférence sur les donnés d'*entraînement*

In [ ]:
les_images = [] 
etiquettes_vraies = []  
etiquettes_predites = [] 
liste_noms_classes = donnees_entrainement.class_names
print("Noms de classes:\n",liste_noms_classes)
print()
# boucler sur le jeu de données d'entraînement
for lot_images, lot_etiquettes in donnees_entrainement_normalisees: 
   # accumuler les images
   les_images.append(lot_images)
   # accumuler les vraies étiquettes
   etiquettes_vraies.append(lot_etiquettes)
   # faire des prédictions
   predictions = modele_de_transfert.predict(lot_images)
   # accumuler les étiquettes prédites
   etiquettes_predites.append(np.argmax(predictions, axis = - 1))
# convertir les listes d'étiquettes en tenseurs
liste_vraies_etiquettes_entrainement = tf.concat([item+1 for item in etiquettes_vraies], axis = 0)
liste_etiquettes_predites_entrainement = tf.concat([item+1 for item in etiquettes_predites], axis = 0)
liste_images = tf.concat([item for item in les_images], axis = 0)
print("Étiquettes prêtes!")


#### Mesure d'exactitude

In [ ]:
from sklearn import metrics

exactitude_test = metrics.accuracy_score(liste_vraies_etiquettes_entrainement, liste_etiquettes_predites_entrainement)
print("Exactitude:   %0.2f" % exactitude_test)


#### Matrice de confusion

In [ ]:
# https://stackoverflow.com/questions/65618137/confusion-matrix-for-multiple-classes-in-python
%matplotlib inline
import matplotlib.pyplot as plt
import itertools

def afficher_matrice_de_confusion(matrice_confusion_brute, classes,
                          normalisation=False,
                          titre='Matrice de confusion',
                          carte_des_couleurs=plt.cm.Blues):
    plt.figure(figsize=(14,12))
    plt.imshow(matrice_confusion_brute, interpolation='nearest', cmap=carte_des_couleurs)
    plt.title(titre)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalisation:
        matrice_confusion_brute = matrice_confusion_brute.astype('float') / matrice_confusion_brute.sum(axis=1)[:, np.newaxis]
        print("Matrice de confusion normalisée")
    else:
        print('Matrice de confusion non normalisée')

    seuil = matrice_confusion_brute.max() / 2.
    for i, j in itertools.product(range(matrice_confusion_brute.shape[0]), range(matrice_confusion_brute.shape[1])):
        plt.text(j, i, matrice_confusion_brute[i, j],
                 horizontalalignment="center",
                 color="white" if matrice_confusion_brute[i, j] > seuil else "black")

    plt.tight_layout()
    plt.ylabel('Vraies étiquettes')
    plt.xlabel('Étiquettes prédites')

print("Afficher_matrice_de_confusion")


In [ ]:
matrice_confusion_brute = metrics.confusion_matrix(liste_vraies_etiquettes_entrainement, liste_etiquettes_predites_entrainement)
afficher_matrice_de_confusion(matrice_confusion_brute, classes=liste_noms_classes)


#### Rapport de classification

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(liste_vraies_etiquettes_entrainement, liste_etiquettes_predites_entrainement, target_names=liste_noms_classes))


#### Examen des erreurs

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

def trouver_nom_arbre(index):
    return noms_arbres[index].split("-")[1]

nombre_erreurs = 0
images_mal_classees = []
fig = plt.figure(figsize=(12,4))
for (index,etiq_vraie,etiq_pred,image) in zip(range(len(liste_images)),
                                              liste_vraies_etiquettes_entrainement,
                                              liste_etiquettes_predites_entrainement,
                                              liste_images):
    etiq_pred = etiq_pred.numpy()
    etiq_vraie = etiq_vraie.numpy()
    if (etiq_vraie != etiq_pred):
        print("_"*80)
        print("etiq_pred:",etiq_pred,"- etiq_vraie:",etiq_vraie,)
        print("*** ERREUR*** Prédiction:",etiq_pred,trouver_nom_arbre(etiq_pred),"- Vraie:",etiq_vraie,trouver_nom_arbre(etiq_vraie))
        chemin_image_originale = donnees_entrainement.file_paths[index]
        print("Chemin image originale:",chemin_image_originale)
        images_mal_classees.append(chemin_image_originale)
        image_originale = mpimg.imread(chemin_image_originale)
        plt.axis('Off')
        plt.subplot(1,2,1)
        plt.imshow(image_originale)
        plt.subplot(1,2,2)
        plt.imshow(image)
        nombre_erreurs += 1
        plt.show()
print("_"*80)
print("Nombre total d'erreurs:",nombre_erreurs)


In [ ]:
print("Fin de l'exécution du carnet IPython")